In [2]:
import requests
import json
import pandas as pd
# Baba's id
testid=330626



In [ ]:
# testid=13025
# Above works for 2017
# each year has a specific ID



# Obtaining id's

- the cache id isn't device dependent

In [ ]:
zipcode = 75227
zipcode_URL=f'https://openpaymentsdata.cms.gov/api/1/datastore/query/d0ca4ed9-0a67-5e9e-9a08-fab7b975d61c?keys=true&limit=100&offset=0&conditions%5B0%5D%5Bproperty%5D=covered_recipient_profile_zipcode&conditions%5B0%5D%5Bvalue%5D={zipcode}%25&conditions%5B0%5D%5Boperator%5D=LIKE&sorts%5B0%5D%5Bproperty%5D=covered_recipient_profile_last_name&sorts%5B0%5D%5Border%5D=asc&sorts%5B1%5D%5Bproperty%5D=covered_recipient_profile_first_name&sorts%5B1%5D%5Border%5D=asc&sorts%5B2%5D%5Bproperty%5D=covered_recipient_profile_middle_name&sorts%5B2%5D%5Border%5D=asc'
dump = json.loads(requests.get(zipcode_URL).text)
id = dump['results'][2]['covered_recipient_profile_id']
print(dump)

In [ ]:
limit = 500
state ='TX'
doctorDF=0
for i in range(202):
    offset =i*limit
    doctorListURL=f'https://openpaymentsdata.cms.gov/api/1/datastore/query/d0ca4ed9-0a67-5e9e-9a08-fab7b975d61c?keys=true&limit={limit}&offset={offset}&conditions%5B0%5D%5Bproperty%5D=covered_recipient_profile_state&conditions%5B0%5D%5Bvalue%5D={state}&conditions%5B0%5D%5Boperator%5D=%3D&sorts%5B0%5D%5Bproperty%5D=covered_recipient_profile_last_name&sorts%5B0%5D%5Border%5D=asc&sorts%5B1%5D%5Bproperty%5D=covered_recipient_profile_first_name&sorts%5B1%5D%5Border%5D=asc&sorts%5B2%5D%5Bproperty%5D=covered_recipient_profile_middle_name&sorts%5B2%5D%5Border%5D=asc'
    newDoctorList=(json.loads(requests.get(doctorListURL).text))['results']
    newDF=pd.DataFrame.from_dict(newDoctorList)[['covered_recipient_profile_primary_specialty','covered_recipient_profile_id','covered_recipient_npi','covered_recipient_profile_zipcode','covered_recipient_profile_first_name','covered_recipient_profile_last_name','covered_recipient_profile_type']]
    if i == 0:
        doctorDF = newDF
    else:
        doctorDF=doctorDF.append(newDF)

In [ ]:
doctorDF.reset_index(inplace = True)
del doctorDF['index']
doctorDF

In [ ]:
doctorDF.reset_index(inplace = True)
del doctorDF['index']
doctorDF.to_csv('Texas Doctor Data.csv')

# Getting funding Sources

In [ ]:
# each year has a specific id that is passed as a token when requesting data on payments
year_ids={
    2015:'3526a13d-5d1c-5de1-bdc7-d298703270cf',
    2016:'47fcc5d3-e7f7-5b49-870e-44f002f8cdcf',
    2017:'eeca1e54-a309-5a62-8203-9b88de15f31f',
    2018:'9dfd9047-64b2-5b52-8fe2-48f280859693',
    2019:'3ac03130-dafb-5a7b-89fb-2b2790c05f26',
    2020:'93d9435c-cf1f-56fb-83f4-71824fbdf8ec',
    2021:'7dfb0fb2-e573-5448-a968-67899434ce8b'
    }


In [2]:
def doctorSearch(df):
    df=df[['covered_recipient_profile_primary_specialty','covered_recipient_profile_id','covered_recipient_profile_zipcode']]
    df[['covered_recipient_profile_zipcode']]=df['covered_recipient_profile_zipcode'].split('-')[0]

    id=df['covered_recipient_profile_id']
    paymentDump=[]
    for year in range(2015,2022):
        token=year_ids[year]
        doctor_page_url=f'https://openpaymentsdata.cms.gov/api/1/datastore/query/{token}?keys=true&limit=100&offset=0&conditions%5B0%5D%5Bproperty%5D=covered_recipient_profile_id&conditions%5B0%5D%5Bvalue%5D={id}&conditions%5B0%5D%5Boperator%5D=%3D&sorts%5B0%5D%5Bproperty%5D=total&sorts%5B0%5D%5Border%5D=desc&properties%5B0%5D%5Bresource%5D=t&properties%5B0%5D%5Bproperty%5D=applicable_manufacturer_or_applicable_gpo_making_payment_name&properties%5B1%5D%5Bresource%5D=t&properties%5B1%5D%5Bproperty%5D=nature_of_payment_or_transfer_of_value&properties%5B2%5D%5Bresource%5D=t&properties%5B2%5D%5Bproperty%5D=date_of_payment&properties%5B3%5D%5Bresource%5D=t&properties%5B3%5D%5Bproperty%5D=applicable_manufacturer_or_applicable_gpo_making_payment_id&properties%5B4%5D%5Bresource%5D=t&properties%5B4%5D%5Bproperty%5D=record_id&properties%5B5%5D%5Balias%5D=total&properties%5B5%5D%5Bexpression%5D%5Boperator%5D=%2B&properties%5B5%5D%5Bexpression%5D%5Boperands%5D%5B0%5D%5Bresource%5D=t&properties%5B5%5D%5Bexpression%5D%5Boperands%5D%5B0%5D%5Bproperty%5D=total_amount_of_payment_usdollars&properties%5B5%5D%5Bexpression%5D%5Boperands%5D%5B1%5D=0'
        # doctor_page_url=f'https://openpaymentsdata.cms.gov/api/1/datastore/query/{token}?keys=true&limit=100&offset=0&conditions[0][property]=covered_recipient_profile_id&conditions[0][value]={id}&conditions[0][operator]==&sorts[0][property]=total&sorts[0][order]=desc&properties[0][resource]=t&properties[0][property]=applicable_manufacturer_or_applicable_gpo_making_payment_name&properties[1][resource]=t&properties[1][property]=nature_of_payment_or_transfer_of_value&properties[2][resource]=t&properties[2][property]=applicable_manufacturer_or_applicable_gpo_making_payment_id&properties[3][alias]=total&properties[3][expression][operator]=+&properties[3][expression][operands][0][resource]=t&properties[3][expression][operands][0][property]=total_amount_of_payment_usdollars&properties[3][expression][operands][1]=0'
        loaded=json.loads(requests.get(doctor_page_url).text)
        if 'status' not in loaded:
            paymentDump =paymentDump + loaded['results']
    # create a dataframe out of the payments
    # I need to copy all the data from the df dataframe across the rows of the new dataframe
    paymentDF=pd.DataFrame.from_dict(paymentDump)

    paymentDF[df.index]=df.values
    # cloning the data from the doctor's general info across the payment dataframe
    # combinedDF=pd.merge(df,paymentDF,how='right')

    return paymentDF



In [ ]:
doctorSearch(doctorIDs.loc[2])

In [3]:
doctorIDs=pd.read_csv('Texas Doctor Data.csv')
doctorIDs.drop(doctorIDs.columns[[0]],axis=1,inplace=True)
zipcodeList=doctorIDs['covered_recipient_profile_zipcode'].unique().tolist()
fails = 0
# for zipcode in zipcodeList:
#     mask =doctorIDs[doctorIDs['covered_recipient_profile_zipcode']==zipcode]
#     pings=mask[mask['covered_recipient_profile_primary_specialty'].str.contains(r'Family|Pediatrics',na=False)].count()[0]
#     if pings==0:
#         fails+=1
#         print(mask['covered_recipient_profile_primary_specialty'])
# print(fails)
# print(len(zipcodeList))
# Ditching sorting by physician type to avoid introducing bias and imbalence into data

In [26]:
import random
sampleDF = pd.DataFrame()
sampledZipcodes = random.sample(zipcodeList,k=250)
for zipcode in sampledZipcodes:
    mask =doctorIDs[doctorIDs['covered_recipient_profile_zipcode']==zipcode]
    if mask.count()[0]>=3:
        mask = mask.sample(n=3)
    
    sampleDF = sampleDF.append(mask,ignore_index=True)


sampleDF
# Capped 3 doctors max per zipcode, 250 zipcodes picked at random
sampleDF.to_csv('SampledDoctorData.csv')

In [9]:
sampleDF

""


In [97]:
doctorIDs
# Get rows with a certain zipcode in them
# From those rows only take doctors with a certain specialty, ideally gen practice
mask =doctorIDs[doctorIDs['covered_recipient_profile_zipcode']==75224]
mask[mask['covered_recipient_profile_primary_specialty'].str.contains(r'Family|Pediatrics',na=False)].count()[0]

22

dict_keys([77030, 78229, 76104, 75390, 75231, 75246, 75093, 76508, 75240, 77555, 78234, 78705, 75701, 75235, 77024, 77042, 79106, 78258, 77479, 75230, 79902, 77054, 77598, 77380, 77074, 78731, 78539, 78759, 78411, 75034, 78550, 78758, 78207, 76712, 75503, 77478, 77070, 79410, 79905, 78236, 76132, 77304, 78746, 78216, 76107, 78240, 75024, 78613, 78503, 78130, 77025, 77090, 75075, 78681, 77339, 79430, 77401, 77584, 77802, 77494, 78665, 77008, 78041, 77384, 78251, 77845, 76063, 76092, 76051, 75013, 78745, 78701, 75605, 75601, 77450, 77429, 79761, 79601, 77027, 78404, 77521, 79912, 77338, 78212, 78704, 78504, 75204, 77375, 78723, 75080, 76012, 77057, 75203, 77004, 77379, 79701, 75904, 78028, 77702, 78414, 76301, 77504, 79936, 76028, 75208, 76201, 79424, 75237, 76903, 75070, 75150, 77065, 76109, 77002, 79925, 77901, 75071, 77098, 78232, 75234, 76015, 75028, 77089, 79606, 75032, 75243, 75035, 78666, 75082, 75061, 78756, 75010, 75219, 75251, 75703, 78217, 77005, 77058, 76544, 77082, 77055, 75

In [ ]:
# for doctor in doctorIDs['covered_recipient_profile_id']:
#     doctorSearch(doctor)

recordsDF = ''
for i in range(0,5):
    if i%100==0:
        print(i)
    if i==0:
        recordsDF = doctorSearch(doctorIDs.loc[i])
    else:
        recordsDF=recordsDF.append(doctorSearch(doctorIDs.loc[i]))
    

# The new combined dataframe should mark the transactions, their source zipcode,  their professions,and ids




In [ ]:
recordsDF
recordsDF.to_csv('Texas Records Data.csv')